# Define some paths

In [ ]:
# The base path where all outputs are stored. You can create a folder or point to an existing accessible location.
YOUR_BASE_PATH='shared_fs'
CHECKPOINT_STORAGE="/run/determined/workdir/shared_fs/checkpoints"

# The following paths will be added to the base path (e.g. audio files will be stored in 'shared_fs/audio_files'). 
AUDIO_PATH='audio_schmidt' # where audio files are downloaded
SNIPPETS_PATH='schmidt' # where 10s clips are stored

# Only populate one of the following: either a YouTube playlist or a single video. Make sure the video doesn't have an age restriction, or you will have to log in
PLAYLIST_URL='' #
SINGLE_VIDEO_URL='https://www.youtube.com/watch?v=pvh_9HAlDtM' 

: 

# Audio Downloading + Model Training Installations

In [9]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 3316 kB of archives.
After this operation, 11.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 git-lfs amd64 2.9.2-1 [3316 kB]
Fetched 3316 kB in 1s (4614 kB/s)  
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package git-lfs.
(Reading database ... 61151 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.9.2-1_amd64.deb ...
Unpacking git-lfs (2.9.2-1) ...
Setting up git-lfs (2.9.2-1) ...


In [10]:
!git clone https://huggingface.co/fnlp/SpeechTokenizer
!pip3 install speechtokenizer

Cloning into 'SpeechTokenizer'...
remote: Enumerating objects: 17, done.
remote: Total 17 (delta 0), reused 0 (delta 0), pack-reused 17
Unpacking objects: 100% (17/17), 348.04 KiB | 7.73 MiB/s, done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 95.8 MB/s eta 0:00:00


In [11]:
!pip install datasets
!pip install mamba-ssm==1.1.0
!pip install huggingface_hub
!pip install torchinfo
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 130.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 153.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 133.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 177.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 131.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 135.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 135.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 139.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━

In [12]:
!pip install pytube
!pip install moviepy
!pip install soundfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.3/313.3 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 166.8 MB/s eta 0:00:00a 0:00:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110721 sha256=75bae7ee3b8d73f50bd725bdec10402906ff06c08de9235c1d3f47cdb32c1d37
  Stored in directory: /tmp/pip-ephem-wheel-cache-6jymesd9/wheels/96/32/2d/e10123bd88fbfc02fed53cc18c80a171d3c87479ed845fa7c1
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.4 MB/s eta 0:00:00a 0:00:01


# Define some tokenization functions

In [13]:
from speechtokenizer import SpeechTokenizer
import soundfile as sf
import torchaudio
import torch
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

NUM_QUANTIZERS_USED = 4
batch_size = 4

config_path = 'SpeechTokenizer/speechtokenizer_hubert_avg/config.json'
ckpt_path = 'SpeechTokenizer/speechtokenizer_hubert_avg/SpeechTokenizer.pt'
speech_tokenizer = SpeechTokenizer.load_from_checkpoint(config_path, ckpt_path).to(device)
speech_tokenizer.eval()


def normalize_waveform(waveform, sr):
    if len(waveform.shape) == 2 and waveform.shape[1] > 0:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    waveform = waveform.reshape(1, -1)
    waveform = torchaudio.functional.resample(waveform, sr, speech_tokenizer.sample_rate)
    return waveform


# Expected input: waveform of shape (B, C, V)
# The encoding returns shape (num codebooks, 1, timestep). Using it with batch size > 1 gave me a lot of errors so I didn't try it.
# After encoding the tokens, I threw away some of the later codebooks which encode less information before I flattened them as explained below.
def tokenize(waveform):
    with torch.no_grad():
        codes = speech_tokenizer.encode(waveform.unsqueeze(0).to(device))  # codes: (n_q, B, T)
    semantic_tokens = codes[:NUM_QUANTIZERS_USED, :, :].cpu()
    return flatten_tokens(semantic_tokens)

def save_waveform(filename, waveform):
  torchaudio.save(filename, waveform[0].detach().cpu(), 16000)

def decode_tokens(tokens):
  unflattened_tokens = unflatten_tokens(tokens)
  return speech_tokenizer.decode(unflattened_tokens)

def save_to_file(tok, filename):
  outputwav = decode_tokens(tok.detach().to(device))
  save_waveform(filename, outputwav)

# Transposing the timestep and code books before flattening to have it be a1, b1, c1 instead of a1, a2, a3, b1, b2, b3,
# since I'm throwing away some of the codebooks, and I also want to be able to generate based on timestep
def flatten_tokens(tokens):
    n_q, B, T = tokens.shape
    transpose_tokens = tokens.transpose(0, 2)
    return transpose_tokens.reshape(B, T * NUM_QUANTIZERS_USED)


def unflatten_tokens(tokens):
    B, L = tokens.shape
    T = L // NUM_QUANTIZERS_USED
    return tokens.reshape(T, B, NUM_QUANTIZERS_USED).transpose(0, 2)

# Download Audio

In [26]:
from pytube import Playlist, YouTube

def download_audio_from_playlist(playlist_url, output_path):
    playlist = Playlist(playlist_url)
    for video in playlist.videos:
        audio_stream = video.streams.get_audio_only()
        audio_stream.download(output_path=output_path, filename=video.title + ".mp4")

def download_audio_from_video(video_url, output_path):
    video = YouTube(video_url)
    audio_stream = video.streams.get_audio_only()
    audio_stream.download(output_path=output_path, filename=video.title + ".mp4")
    
playlist_url = PLAYLIST_URL
video_url = SINGLE_VIDEO_URL

if playlist_url:
    download_audio_from_playlist(playlist_url, f"{YOUR_BASE_PATH}/{AUDIO_PATH}")
elif video_url:
    download_audio_from_video(video_url, f"{YOUR_BASE_PATH}/{AUDIO_PATH}")

# Install audio processing tools

In [29]:
!apt-get update
!apt-get install -y ffmpeg

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 https://packages.cloud.google.com/apt cloud-sdk InRelease [6361 B]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease    
Get:6 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [582 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [3144 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [32.5 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3770 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3293 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1165 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [29.7 kB]
Get:13 http://archive.ubuntu.com/ubuntu foc

In [30]:
!ffmpeg -version

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enabl

# Split audio into 10s snippets

In [31]:
import subprocess
import os

def extract_and_split_audio(mp4_file, output_dir, clip_length=10):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    base_name = os.path.splitext(os.path.basename(mp4_file))[0]
    output_format = "wav"
    temp_audio_file = os.path.join(output_dir, f"{base_name}_temp.{output_format}")

    # Extract the audio from the video, suppressing error messages
    with open(os.devnull, 'w') as FNULL:
        subprocess.run(["ffmpeg", "-i", mp4_file, "-q:a", "0", "-map", "a", temp_audio_file], 
                       stdout=FNULL, stderr=subprocess.STDOUT, check=True)

    # Get the duration of the audio file
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries", 
                             "format=duration", "-of", 
                             "default=noprint_wrappers=1:nokey=1", temp_audio_file], 
                            text=True, capture_output=True)
    total_duration = float(result.stdout)

    # Split the audio file into chunks
    for start in range(0, int(total_duration), clip_length):
        end = min(start + clip_length, int(total_duration))
        output_file = os.path.join(output_dir, f"{base_name}_clip_{start}_{end}.{output_format}")
        with open(os.devnull, 'w') as FNULL:
            subprocess.run(["ffmpeg", "-y", "-i", temp_audio_file, "-ss", str(start), "-to", 
                            str(end), "-c", "copy", output_file], 
                           stdout=FNULL, stderr=subprocess.STDOUT, check=True)

    # Remove the temporary audio file
    os.remove(temp_audio_file)

def process_all_files(input_dir, output_dir):
    for file in os.listdir(input_dir):
        if file.endswith('.mp4'):
            mp4_file_path = os.path.join(input_dir, file)
            extract_and_split_audio(mp4_file_path, output_dir)

In [32]:
input_directory = f"{YOUR_BASE_PATH}/{AUDIO_PATH}" # Folder where your MP4 files are located
output_directory = f"{YOUR_BASE_PATH}/{SNIPPETS_PATH}"  # Folder where you want to save the clips

process_all_files(input_directory, output_directory)

# Normalize + tokenize the waveforms

In [36]:
!pip install librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/253.7 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 135.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.8/530.8 kB 142.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 184.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 201.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 157.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 MB 258.1 MB/s eta 0:00:00a 0:00:01


In [33]:
!mkdir shared_fs/testfiles

mkdir: cannot create directory ‘shared_fs/testfiles’: File exists


In [37]:
from datasets import load_dataset
import torch
import numpy as np


print("Loading Dataset")
# Select subset from 10s clips
audio_dataset = load_dataset("audiofolder", data_dir=output_directory)["train"]

print(audio_dataset)


print("Normalizing the waveforms")
audio_dataset = audio_dataset.map(
    lambda x: {
        "original_sampling_rate": x["audio"]["sampling_rate"],
        "audio_array": normalize_waveform(
            torch.tensor(x["audio"]["array"]), x["audio"]["sampling_rate"]
        ),
    },
    remove_columns=["audio"],
    # keep_in_memory=True,
    writer_batch_size=15000,
)

print(audio_dataset)


print("Making sure the dataset is in the correct format")

def standardize_audio_length_with_tolerance(example, expected_length):
    audio_array = example["audio_array"]
    if not isinstance(audio_array, torch.Tensor):
        audio_array = torch.tensor(audio_array)

    # Assuming audio_array is 2D: [channels, length]
    current_length = audio_array.shape[1]

    if current_length < expected_length:
        # Pad with zeros. The padding size should match the number of channels
        padding_length = expected_length - current_length
        padding = torch.zeros((audio_array.shape[0], padding_length), dtype=audio_array.dtype)
        audio_array = torch.cat([audio_array, padding], dim=1)
    elif current_length > expected_length:
        # Trim the excess length
        audio_array = audio_array[:, :expected_length]

    return {"audio_array": audio_array}

# Apply the function
audio_dataset = audio_dataset.map(
    lambda x: standardize_audio_length_with_tolerance(x, expected_length=160000),
    batched=False
)


print(audio_dataset)

print("Tokenizing the waveforms")
audio_dataset = audio_dataset.map(
    lambda x: {"tokens": tokenize(torch.tensor(x["audio_array"]))
               },
    remove_columns=[
        "audio_array",
    ],
    writer_batch_size=15000,
)

print(audio_dataset)

# Checking the files to see if the tokenization worked correctly.
for idx, t in enumerate(audio_dataset.select(range(0, 10))):
    save_to_file(torch.tensor(t["tokens"]).to(device), f"{YOUR_BASE_PATH}/testfiles/{idx}_test.wav")

Loading Dataset


Resolving data files:   0%|          | 0/59 [00:00<?, ?it/s]

Dataset({
    features: ['audio'],
    num_rows: 59
})
Normalizing the waveforms


Map:   0%|          | 0/59 [00:00<?, ? examples/s]

Dataset({
    features: ['original_sampling_rate', 'audio_array'],
    num_rows: 59
})
Making sure the dataset is in the correct format


Map:   0%|          | 0/59 [00:00<?, ? examples/s]

Dataset({
    features: ['original_sampling_rate', 'audio_array'],
    num_rows: 59
})
Tokenizing the waveforms


Map:   0%|          | 0/59 [00:00<?, ? examples/s]

Dataset({
    features: ['original_sampling_rate', 'tokens'],
    num_rows: 59
})


# Create train/test dataloaders

In [38]:
print(audio_dataset)

audio_dataset = audio_dataset.with_format('torch')
audio_dataset = audio_dataset.train_test_split(0.05)

# Setting up the train and test dataloader
train_dataloader = torch.utils.data.DataLoader(audio_dataset['train'], batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(audio_dataset['test'], batch_size=batch_size, shuffle=True)

Dataset({
    features: ['original_sampling_rate', 'tokens'],
    num_rows: 59
})


# Define model + training functions

In [48]:
from tqdm import tqdm
from mamba_ssm import Mamba
import matplotlib.pyplot as plt
import numpy as np
import torchinfo
import torchaudio
from torch.utils.data import DataLoader
import os
import torch
import torch.nn as nn
from torch.nn import  functional as F
#hyperparams

epochs = 20
lr = 1e-3
block_size = 2000
device = "cuda" if torch.cuda.is_available() else "cpu"
max_iters = 10000
print_iters = 100
eval_iters = 10
eval_interval = 300
n_embed=384
n_heads = 6
n_layers = 6
dropout = 0.2
vocab_size=1024
from tqdm import tqdm

# ---------




class SelfAttentionHead(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.keys = nn.Linear(n_embed, head_size)
    self.queries = nn.Linear(n_embed, head_size)
    self.values = nn.Linear(n_embed, head_size)
    self.head_size = head_size
    self.n_embed = n_embed
    self.register_buffer('tril', torch.tril(torch.ones((block_size,block_size))).to(device))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.keys(x) # (B,T,C_h)
    q = self.queries(x) # (B,T,C_h)
    v = self.values(x) # (B,T,C_h)
    wei = k @ q.transpose(-1,-2) * C**(-0.5)# (B,T,T)
    wei = wei.masked_fill( self.tril[:T,:T]==0, float('-inf'))
    # wei = F.softmax(wei, dim=-1) # (B,T,T)
    wei = torch.log(torch.exp(wei)+1) # (B,T,T)
    wei = self.dropout(wei)
    out = wei @ v # (B,T,C_h)
    return out


class MultiHeadAttention(nn.Module):
  def __init__(self, n_heads, head_size) -> None:
    super().__init__()
    self.heads = nn.ModuleList([SelfAttentionHead(head_size) for _ in range(n_heads)])
    self.proj = nn.Linear(n_embed, n_embed)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    out = torch.cat([head(x) for head in self.heads], dim=-1)
    out = self.proj(out)
    out = self.dropout(out)
    return out

class FeedForward(nn.Module):
  def __init__(self, n_embed) -> None:
    super().__init__()
    self.ffn = nn.Sequential(
      nn.Linear(n_embed, 4*n_embed),
      nn.ReLU(),
      nn.Linear(4*n_embed, n_embed),
      nn.Dropout(dropout),
    )
  def forward(self, x):
    return self.ffn(x)

class Block(nn.Module):
  def __init__(self, n_embed, n_heads) -> None:
    super().__init__( )
    self.head_size = n_embed // n_heads
    # self.sa_head = MultiHeadAttention(n_heads, self.head_size)
    self.sa_head = Mamba(
      # This module uses roughly 3 * expand * d_model^2 parameters
      d_model=n_embed, # Model dimension d_model
      d_state=16,  # SSM state expansion factor
      d_conv=4,    # Local convolution width
      expand=1,    # Block expansion factor
  ).to("cuda")
    self.ffn = FeedForward(n_embed)
    self.ln1 = nn.LayerNorm(n_embed)
    self.ln2 = nn.LayerNorm(n_embed)


  def forward(self, x):
    x = x + self.sa_head(self.ln1(x))
    x = x + self.ffn(self.ln2(x))

    return x

class MambaAudioModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,n_embed)
    self.position_embedding_table = nn.Embedding(block_size,n_embed)
    self.lm_head = nn.Linear(n_embed,vocab_size)
    self.ffn = FeedForward(n_embed)
    print("layers", n_layers)
    self.blocks = nn.Sequential(*[Block(n_embed,n_heads=n_heads) for _ in range(n_layers)])


  def forward(self, idx, targets=None):
    # idx = idx[:,-block_size:]
    B,T = idx.shape
    tok_emb = self.token_embedding_table(idx) # (B,T, C_e)
    pos_emb = self.position_embedding_table(torch.arange(T,device=device)) # (T, C_e)
    x = tok_emb + pos_emb # (B,T,Q, C_e)
    x = self.blocks(x) # (B,T,Q, C_e)
    logits = self.lm_head(x) # (B,T,vocab_size)
    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
      logits = logits.view(B,T,C)
    return logits, loss


def estimate_test_loss(model, dataset):
  model.eval()
  test_losses = []
  with torch.no_grad():
    for tokens in tqdm(dataset['test']['tokens']):
      tokens = tokens.to(device)
      x = tokens[:,:-1].contiguous()
      y = tokens[:,1:].contiguous()
      logits, loss = model(x,y )
      test_losses.append(loss)
  model.train()
  return sum(test_losses)/len(test_losses)

# Install Determined and verify

In [46]:
!pip install --upgrade determined
!pip install tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 141.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 124.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 kB 122.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 203.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 135.5 MB/s eta 0:00:00


In [41]:
import determined as det
!det --version

det 0.27.0


# Train model using Detached Mode (report metrics and store checkpoints in Determined)

In [ ]:
import torchinfo
from tqdm.notebook import tqdm
import random
from determined.experimental import core_v2


# initialize core context
core_v2.init(
    defaults=core_v2.DefaultConfig(
        name="schmidt",
        checkpoint_storage=CHECKPOINT_STORAGE,
    ),
    
    unmanaged=core_v2.UnmanagedConfig(
        external_experiment_id="2",
    ),
)

    
# initialize model 
model = MambaAudioModel(vocab_size).to(device)
optimizer = torch.optim.AdamW(model.parameters(),lr=lr)


# load checkpoint helper function
def load_state(checkpoint_directory, trial_id):
    checkpoint_directory = pathlib.Path(checkpoint_directory)

    with checkpoint_directory.joinpath("checkpoint.pt").open("rb") as f:
        model = torch.load(f)
    with checkpoint_directory.joinpath("state").open("r") as f:
        epochs_completed, ckpt_trial_id = [int(field) for field in f.read().split(",")]

    if ckpt_trial_id != trial_id:
        epochs_completed = 0

    return model, epochs_completed


initial_i = 0

# print model + put in train mode
print(torchinfo.summary(model))
model.train()


# training loop 
ind = initial_i

for epoch in tqdm(range(epochs)):
    for batch in tqdm(train_dataloader):
        
        checkpoint_metadata_dict = {"steps_completed": ind}

        if batch['tokens'].shape[-1] != block_size:
            continue
        batch_size = batch['tokens'].shape[0]
        tokens = batch['tokens'].to(device).reshape(batch_size,block_size)

        x = tokens[:,:-1].contiguous()
        y = tokens[:,1:].contiguous()
        logits, loss = model(x,y)

        if ind % 5 == 0:
            print(loss)
            core_v2.train.report_training_metrics(steps_completed=ind, metrics={"loss": loss.item()})

        if ind % 10 == 0:
            tl = estimate_test_loss(model, audio_dataset)
            print("testloss", tl)
            core_v2.train.report_validation_metrics(steps_completed=ind, metrics={"loss": tl.item()})


        if ind % 100 == 0:
            with core_v2.checkpoint.store_path(checkpoint_metadata_dict) as (path, storage_id):
                torch.save(model.state_dict(), path / "checkpoint.pt")
                with path.joinpath("state").open("w") as f:
                    f.write(f"{epochs},{core_v2.info.trial.trial_id}")
                    
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        
        # Prevents gradient explosion.
        torch.nn.utils.clip_grad.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        ind += 1

           
checkpoint_metadata_dict = {"steps_completed": ind}
with core_v2.checkpoint.store_path(checkpoint_metadata_dict) as (path, storage_id):
    torch.save(model.state_dict(), path / "checkpoint.pt")
    with path.joinpath("state").open("w") as f:
        f.write(f"{epoch},{core_v2.info.trial.trial_id}")
                
core_v2.close()
print(torchinfo.summary(model))

: 

# Test out model

In [ ]:
# load checkpoint - replace checkpoint path to that of the desired checkpoint
model = MambaAudioModel(vocab_size).to(device)
model.load_state_dict(torch.load(f"{YOUR_BASE_PATH}/checkpoints/ec795afe-64dd-42e0-a0e0-a36ab3aeeb86/checkpoint.pt"))

: 

In [ ]:
# Generate new audio
def unconditional_generation(model):
    idx = torch.tensor([[10,]]).to(device)
    max_new_tokens = 1999
    idx_next = []
    for i in tqdm(range(max_new_tokens)):
        idx_cond = idx[:,-block_size:]
        logits, loss = model(idx_cond)
        last_timestep = logits[:,-1,:]
        probs = F.softmax(last_timestep, dim=1)
        next_index = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, next_index), dim=1)

    save_to_file(idx, f"{YOUR_BASE_PATH}/unconditional_test.wav")

unconditional_generation(model)

: 